# Quant Course: Lesson 3
## Building a pricing library - Initial steps

## What did we learn last time?

<b>Risk-free interest rate</b> ($r$): Annual return of a riskless investment, i.e. the minimum return an investor expects for any investment.

<b>Spot contract</b>: An agreement between two parties (a buyer and a seller) to trade an asset "*immediately*".

Fair value: $ S_t $

The buyer of a contract takes a *Long* position, while the seller of a contract takes a *Short* position.

<b>Forward contract</b>: An agreement between two parties (a buyer and a seller) to trade an underlying asset at a <u>certain price</u> ($K$: strike) on a <u>certain future date</u> ($T$: expiry). This is an <u>obligation</u> to both parties.

Payoff at expiry (LF): $ \;\;\; S_T-K $

<img src="./img/payoff_forward.jpg" style="margin-left:auto; margin-right:auto; width: 730px;" />

<b>(European) Call option contract</b>: The option buyer has the <u>right</u>, but not the obligation, to <u><b>buy</b></u> an underlying asset at a certain price ($K$: strike) on a certain future date ($T$: expiry). If option is exercised by buyer, the option seller must <u>sell</u> the underlying asset.

Payoff at expiry (LC): $ \;\;\; max\lbrace S_T-K, 0\rbrace $

<img src="./img/payoff_call.jpg" style="margin-left:auto; margin-right:auto; width: 730px;" />

<b>(European) Put option contract</b>: The option buyer has the <u>right</u>, but not the obligation, to <u><b>sell</b></u> an underlying asset at a certain price ($K$: strike) on a certain future date ($T$: expiry). If option is exercised by the option buyer, the option seller must <u>buy</u> the underlying asset.

Payoff at expiry (LP): $ \;\;\; max\lbrace K-S_T, 0\rbrace $

<img src="./img/payoff_put.jpg" style="margin-left:auto; margin-right:auto; width: 730px;" />

## Pricing under Black-Scholes model assumptions
Spot price: $ S_t $, strike price: $ K $, expiry: $ T $, volatility of underlying: $ \sigma $, risk-free rate: $ r $

<b>Forward (LF)</b>: $ \;\;\; f_t = S_t - Ke^{-r(T-t)} $

<b>Call option (LC)</b>: $ \;\;\; c_t = S_tN(d_1) - Ke^{-r(T-t)}N(d_2) $

<b>Put option (LP)</b>: $ \;\;\; p_t = Ke^{-r(T-t)}N(-d_2) - S_tN(-d_1) $

where $N(\cdot)$ is the standard normal CDF

$ d_1 = \dfrac{1}{\sigma\sqrt{T-t}}\left[ln\left(\dfrac{S_t}{K}\right)+\left(r+\dfrac{\sigma^2}{2}\right)(T-t)\right] \;\;\;\;\;\;\;\;\;\; d_2 = d_1 - \sigma\sqrt{T-t} $

<br>
<b>Put-call parity</b>: $ LF=LC+SP $

$ f_t=c_t-p_t $

## Warm up exercise

Let's price a real life European call option contract on a Google stock using Black-Scholes model.

Data:

* https://finance.yahoo.com/quote/GOOG/options?p=GOOG
* https://www.marketwatch.com/investing/bond/tmubmusd01y

<img src="./img/pricing_example.jpg" style="margin-left:auto; margin-right:auto" />

In [ ]:
import numpy as np
from scipy.stats import norm

def black_scholes_european_call(S0: float, K: float, T: float, sigma: float, r: float):
    d1 = ( np.log( S0 / K ) + ( r + 0.5 * sigma**2 ) * T ) / ( sigma * T**0.5 )
    d2 = d1 - sigma * T**0.5
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    df = np.exp((-1.0)*r*T)
    return S0 * N_d1 - K * df * N_d2

In [ ]:
import datetime as dt

...

premium = ...
print(f'Premium: {round(premium, 2)}$')

## How to build a pricing library?

<b>Goal</b>: Implement a coherent code base that is able to calculate fair value and greeks of various contracts in a generic way.

<b>Requirements</b>:
* Separate market data from pricing code
* Reusability: Avoid code duplication
* Scalability: Should be fairly easy to onboard a new contract or a pricer
* Encapsulation: Bundle related data and funcionalities into a structured unit

<img src="./img/building_blocks_empty.jpg" style="margin-left:auto; margin-right:auto; width: 1000px;" />

<img src="./img/building_blocks_full.jpg" style="margin-left:auto; margin-right:auto; width: 1000px;" />

<img src="./img/contracts.jpg" style="margin-left:auto; margin-right:auto; width: 800px;" />

## Important concepts

<b>Inheritance</b>: Deriving a class (Derived/Child/Sub) from another class (Base/Parent/Super) that share a set of properties and methods.

<b>Abstract Base Class (ABC)</b>: Such a class is not concrete and cannot be instantiated. Instead, it forces derived classes to implement all abstract properties and methods.

<b>Access Modifiers</b>: One can restrict the accessibility of properties and methods to protect internal class data. Access levels: public (default), protected (starts with \_), private (starts with \__)


<b>Let's build the library!</b>

In [ ]:
# Add current folder and QuantCourseBP folder to syspath to import modules
import sys
from pathlib import Path
current = Path(Path().resolve())
sys.path.append(str(current))
sys.path.append(str(current.parents[1]))